In [1]:
import pandas as pd
import numpy as np

weather_aus = pd.read_csv('../data/weather/weatherAUS.csv')
weather_sydney = weather_aus.loc[weather_aus['Location'] == 'Sydney'].copy()
weather_sydney = weather_sydney.dropna()

features = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
            'Sunshine', 'Humidity3pm', 'Pressure9am', 'Pressure3pm',
            'Cloud9am', 'Temp9am', 'Temp3pm', 'RainToday', 'RISK_MM']
feature_dataframe = weather_sydney[features].copy()
feature_dataframe['RainToday'].replace(('Yes', 'No'), (1, 0), inplace=True)

labels_dataframe = weather_sydney['RainTomorrow'].copy()
labels_dataframe.replace(('Yes', 'No'), (1, 0), inplace=True)

weather_data = feature_dataframe.values.astype('float')
labels = labels_dataframe.values.astype('int')

feature_dataframe.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Temp9am,Temp3pm,RainToday,RISK_MM
31168,12.9,20.3,0.2,3.0,10.9,57.0,1028.8,1025.6,3.0,16.9,19.8,0,0.0
31169,13.3,21.5,0.0,6.6,11.0,58.0,1025.9,1022.4,2.0,17.6,21.3,0,0.0
31170,15.3,23.0,0.0,5.6,11.0,63.0,1021.4,1017.8,1.0,19.0,22.2,0,0.0
31174,12.9,26.7,0.2,3.8,12.1,56.0,1018.0,1015.0,1.0,17.8,22.5,0,0.0
31175,14.8,23.8,0.0,6.8,9.6,69.0,1016.0,1014.7,2.0,20.2,20.6,0,1.8


In [2]:
from sklearn.model_selection import train_test_split

train_to_test_ratio = 0.8
X_train, X_test, Y_train, Y_test = train_test_split(weather_data, labels, train_size=train_to_test_ratio)


/home/andreas/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
from scipy.special import expit
from tqdm import tqdm

class BinaryNeuralNetwork:
    def __init__(
        self,
        X_data,
        Y_data,
        epochs=1,
        batch_size=100,
        eta=1e-2,
        lmbd=0.0,
        n_neurons_layer1=100,
        n_neurons_layer2=50,
        n_categories=2,
    ):
        self.X_data_full = X_data
        self.Y_data_full = Y_data

        self.n_inputs = X_data.shape[0]
        self.n_features = X_data.shape[1]
        self.n_neurons_layer1 = n_neurons_layer1
        self.n_neurons_layer2 = n_neurons_layer2
        self.n_categories = n_categories

        self.epochs = epochs
        self.batch_size = batch_size
        self.iterations = self.n_inputs // self.batch_size
        self.eta = eta
        self.lmbd = lmbd

        self.create_biases_and_weights()

    def create_biases_and_weights(self):
        self.weights_layer1 = np.random.randn(self.n_features, self.n_neurons_layer1)
        self.bias_layer1 = np.zeros(self.n_neurons_layer1)

        self.weights_layer2 = np.random.randn(self.n_neurons_layer1, self.n_neurons_layer2)
        self.bias_layer2 = np.zeros(self.n_neurons_layer2)

        self.weights_output = np.random.randn(self.n_neurons_layer2, self.n_categories)
        self.bias_output = np.zeros(self.n_categories)

    def feed_forward(self):
        self.z1 = np.dot(self.X_data, self.weights_layer1) + self.bias_layer1
        self.a1 = expit(self.z1)

        self.z2 = np.dot(self.a1, self.weights_layer2) + self.bias_layer2
        self.a2 = expit(self.z2)

        self.z3 = np.dot(self.a2, self.weights_output) + self.bias_output

        exp_term = np.exp(self.z3)
        self.probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

    def feed_forward_out(self, X):
        z1 = np.dot(X, self.weights_layer1) + self.bias_layer1
        a1 = expit(z1)

        z2 = np.dot(a1, self.weights_layer2) + self.bias_layer2
        a2 = expit(z2)

        z3 = np.dot(a2, self.weights_output) + self.bias_output

        exp_term = np.exp(z3)
        probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)
        return probabilities

    def backpropagation(self):
        error_output = self.probabilities
        error_output[range(self.n_inputs), self.Y_data] -= 1
        error_layer2 = (np.dot(error_output, self.weights_output.T) * self.a2 * (1 - self.a2))
        error_layer1 = (np.dot(error_layer2, self.weights_layer2.T) * self.a1 * (1 - self.a1))

        self.weights_output_gradient = np.dot(self.a2.T, error_output)
        self.bias_output_gradient = np.sum(error_output)

        self.weights_layer2_gradient = np.dot(self.a1.T, error_layer2)
        self.bias_layer2_gradient = np.sum(error_layer2)

        self.weights_layer1_gradient = np.dot(self.X_data.T, error_layer1)
        self.bias_layer1_gradient = np.sum(error_layer1)

        if self.lmbd > 0.0:
            self.weights_output_gradient += self.lmbd * self.weights_output
            self.weights_layer2_gradient += self.lmbd * self.weights_layer2
            self.weights_layer1_gradient += self.lmbd * self.weights_layer1

        self.weights_output -= self.eta * self.weights_output_gradient
        self.bias_output -= self.eta * self.bias_output_gradient
        self.weights_layer2 -= self.eta * self.weights_layer2_gradient
        self.bias_layer2 -= self.eta * self.bias_layer2_gradient
        self.weights_layer1 -= self.eta * self.weights_layer1_gradient
        self.bias_layer1 -= self.eta * self.bias_layer1_gradient

    def predict(self, X):
        probabilities = self.feed_forward_out(X)
        return np.argmax(probabilities, axis=1)

    def predict_probabilities(self, X):
        probabilities = self.feed_forward_out(X)
        return probabilities

    def train(self):
        data_indices = np.arange(self.n_inputs)

        for i in range(self.epochs):
            for j in range(self.iterations):
                chosen_datapoints = np.random.choice(
                    data_indices, size=self.batch_size, replace=False
                )

                self.X_data = self.X_data_full[chosen_datapoints]
                self.Y_data = self.Y_data_full[chosen_datapoints]

                self.n_inputs = self.X_data.shape[0]

                self.feed_forward()
                self.backpropagation()


In [4]:
from sklearn.metrics import accuracy_score

epochs = 5
batch_size = 100
n_neurons_layer1 = 100
n_neurons_layer2 = 50
n_categories = 2

eta_vals = np.logspace(-5, 0, 6)
lmbd_vals = np.logspace(-5, 0, 6)


In [5]:
DNN_numpy = np.zeros((len(eta_vals), len(lmbd_vals)), dtype=object)

for i, eta in enumerate(eta_vals):
    for j, lmbd in enumerate(lmbd_vals):
        dnn = BinaryNeuralNetwork(X_train, Y_train, eta=eta, lmbd=lmbd, epochs=1, batch_size=batch_size,
                                  n_neurons_layer1=n_neurons_layer1, n_neurons_layer2=n_neurons_layer2,
                                  n_categories=n_categories)
        dnn.train()
        
        DNN_numpy[i][j] = dnn
        
        test_predict = dnn.predict(X_test)
        
        print("Learning rate  = ", eta)
        print("Lambda = ", lmbd)
        print("Accuracy score on test set: ", accuracy_score(Y_test, test_predict))
        

Learning rate  =  1e-05
Lambda =  1e-05
Accuracy score on test set:  0.7633136094674556
Learning rate  =  1e-05
Lambda =  0.0001
Accuracy score on test set:  0.7810650887573964
Learning rate  =  1e-05
Lambda =  0.001
Accuracy score on test set:  0.7633136094674556
Learning rate  =  1e-05
Lambda =  0.01
Accuracy score on test set:  0.23668639053254437
Learning rate  =  1e-05
Lambda =  0.1
Accuracy score on test set:  0.23668639053254437
Learning rate  =  1e-05
Lambda =  1.0
Accuracy score on test set:  0.7633136094674556
Learning rate  =  0.0001
Lambda =  1e-05
Accuracy score on test set:  0.7662721893491125
Learning rate  =  0.0001
Lambda =  0.0001
Accuracy score on test set:  0.7633136094674556
Learning rate  =  0.0001
Lambda =  0.001
Accuracy score on test set:  0.23668639053254437
Learning rate  =  0.0001
Lambda =  0.01
Accuracy score on test set:  0.7633136094674556
Learning rate  =  0.0001
Lambda =  0.1
Accuracy score on test set:  0.7899408284023669
Learning rate  =  0.0001
Lambd

/home/andreas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide
